In [15]:
# Pandas importeren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import geopandas as gpd


# Laad de gemeentegrenzen in
gemeenten = gpd.read_file('bestuurlijkegrenzen.gpkg', layer='gemeenten')
gemeenten = gemeenten.set_index('gemeentenaam')

# de dataset van OU inlezen
dataset_OU = pd.read_csv('dataset/COVID-19_aantallen_gemeente_per_dag.csv', header=0, sep=';')
dataset_OU.head()

# de dataset van OU checken: 
# - Date_of_publication, Province, Total_reported, Hospital_admission en Deceased hebben geen missing waarden,
# - Municipality_code en Municipality_name hebben wel missing waarden.
dataset_OU.info()

# dataset_OU['Year'] = dataset_OU['Date_of_publication'].str[:4]
dataset_OU['Date_of_publication'] = pd.to_datetime(dataset_OU['Date_of_publication'])
dataset_OU['Month'] = dataset_OU['Date_of_publication'].dt.month
dataset_OU['Year'] = dataset_OU['Date_of_publication'].dt.year
dataset_OU.head()
# dataset voor 2020 aanmaken
dataset_OU_2020 = dataset_OU[dataset_OU['Year'].astype(str) == '2020']
dataset_OU_2020.head()

# dataset voor 2021 aanmaken
dataset_OU_2021 = dataset_OU[dataset_OU['Year'].astype(str) == '2021']
dataset_OU_2021.head()

# #Laad shapefile in
# SHAPEFILE = 'cbsgebiedsindelingen2023.gpkg'

# cbs_gebiedsindeling = gpd.read_file(SHAPEFILE)
# cbs_gebiedsindeling.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243054 entries, 0 to 243053
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   Date_of_report            243054 non-null  object
 1   Date_of_publication       243054 non-null  object
 2   Municipality_code         226979 non-null  object
 3   Municipality_name         226979 non-null  object
 4   Province                  243054 non-null  object
 5   Security_region_code      242411 non-null  object
 6   Security_region_name      242411 non-null  object
 7   Municipal_health_service  243054 non-null  object
 8   ROAZ_region               239196 non-null  object
 9   Total_reported            243054 non-null  int64 
 10  Hospital_admission        243054 non-null  int64 
 11  Deceased                  243054 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 22.3+ MB


,Date_of_report,Date_of_publication,Municipality_code,Municipality_name,Province,Security_region_code,Security_region_name,Municipal_health_service,ROAZ_region,Total_reported,Hospital_admission,Deceased,Month,Year
116424,2021-12-01 10:00:00,2021-01-01,GM0014,Groningen,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,69,1,1,1,2021
116425,2021-12-01 10:00:00,2021-01-01,GM0034,Almere,Flevoland,VR25,Flevoland,GGD Flevoland,SpoedZorgNet,82,3,0,1,2021
116426,2021-12-01 10:00:00,2021-01-01,GM0037,Stadskanaal,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,18,0,0,1,2021
116427,2021-12-01 10:00:00,2021-01-01,GM0047,Veendam,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,14,0,0,1,2021
116428,2021-12-01 10:00:00,2021-01-01,GM0050,Zeewolde,Flevoland,VR25,Flevoland,GGD Flevoland,SpoedZorgNet,8,0,0,1,2021


In [16]:
# Groepeer de data per provincie en gemeente en tel de aantallen op
temp_dataset = dataset_OU_2020[['Hospital_admission','Deceased','Total_reported','Province', 'Municipality_name', 'Month', 'Year']]

municipality_2020 = temp_dataset.groupby(['Province', 'Municipality_name', 'Month', 'Year'])[['Hospital_admission', 'Deceased', 'Total_reported']].sum().reset_index()
municipality_2020 = pd.melt(municipality_2020, id_vars=['Province','Municipality_name', 'Month', 'Year'], var_name='Type', value_name='Numbers')
# municipality_2020['Year'] = '2020' # Voeg een nieuwe kolom toe voor het jaar
municipality_2020.head(15)

# voer dezelfde stappen uit voor de dataset van 2021
# Groepeer de data per provincie en gemeente en tel de aantallen op
temp_dataset = dataset_OU_2021[['Hospital_admission','Deceased','Total_reported','Province', 'Municipality_name', 'Month', 'Year']]

municipality_2021 = temp_dataset.groupby(['Province', 'Municipality_name', 'Month', 'Year'])[['Hospital_admission', 'Deceased', 'Total_reported']].sum().reset_index()
municipality_2021 = pd.melt(municipality_2021, id_vars=['Province','Municipality_name', 'Month', 'Year'], var_name='Type', value_name='Numbers')
# municipality_2021['Year'] = '2021' # Voeg een nieuwe kolom toe voor het jaar


# creer een dictionary zodat de juiste dataset obv het ingevoerde jaar dynamisch kan worden geselecteerd
datasets = {}
datasets['2020'] = municipality_2020
datasets['2021'] = municipality_2021
datasets['gemeenten'] = gemeenten

corona_per_gemeente = datasets['2020'].groupby(['Municipality_name', 'Month', 'Year'])['Numbers'].sum().reset_index()
corona_per_gemeente.head(20)

,Municipality_name,Month,Year,Numbers
0,'s-Gravenhage,2,2020,0
1,'s-Gravenhage,3,2020,289
2,'s-Gravenhage,4,2020,1234
3,'s-Gravenhage,5,2020,484
4,'s-Gravenhage,6,2020,351
5,'s-Gravenhage,7,2020,302
6,'s-Gravenhage,8,2020,1330
7,'s-Gravenhage,9,2020,3954
8,'s-Gravenhage,10,2020,10070
9,'s-Gravenhage,11,2020,5218


In [17]:
# Creëer een dictionary om de nummers naar maandnamen te mappen
month_names = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'}
    
# de functie om de eerste screenshot van opdracht 2 te tonen
def show_national_graph(year='2020', hue_order=['Total_reported','Hospital_admission','Deceased'], show_months=False):
    
    if show_months == True: 
        national_data = datasets[year].groupby(['Month', 'Type'])['Numbers'].sum().reset_index()
        # Gebruik de map-methode van pandas om de maandnummers te vervangen door de corresponderende maandnamen
        national_data['Month'] = national_data['Month'].map(month_names)
        x_axisData = 'Month'
    else:
        national_data = datasets[year]
        x_axisData = 'Year'
    
    # toon de barplot conform de eerste screenshot van opdracht 2
    fig, ax = plt.subplots(figsize=(10, 5))
    g = sns.barplot(x = x_axisData,
            y = 'Numbers',
            hue = 'Type',
            hue_order=hue_order,
            estimator = np.sum,
            errorbar=None,
            data = national_data)
    
    # de screenshots van opdracht 2 bevatten geen xlabel en ylabel, daarom deze hier ook verwijderen.
    g.set(xlabel='', ylabel='')

    # de eerste screenshot van opdracht 2 heeft een titel, daarom deze hier ook toegevoegd.
    g.set(title='The Netherlands in ' + year)

    # de screenshots van opdracht 2 bevatten ook geen titel voor de legenda, daarom deze hier ook verwijderen.
    plt.legend(title='')

    # de province namen geroteerd voor de leesbaarheid, conform de screenshots van opdracht 2
    plt.xticks(rotation=45)

    plt.show()
    
# de functie om de tweede screenshot van opdracht 2 te tonen
def show_provinces_graph(year='2020', hue_order=['Total_reported','Hospital_admission','Deceased']):
#     province_data = datasets[year].groupby(['Province', 'Type'], numeric_only=True).sum().reset_index()
    
    province_data = datasets[year].groupby(['Province', 'Type', 'Year'])['Numbers'].sum().reset_index()

    # zet de tijdelijke subset om naar province_2020 en past de pandas melt functie toe, ter voorbereiding van de visualisatie
    
    
    # toon de barplot conform de tweede screenshot van opdracht 2
    g = sns.barplot(x = 'Province',
                y = 'Numbers',
                hue = 'Type',
                hue_order=hue_order,
                data = province_data)

    # de screenshots van opdracht 2 bevatten geen xlabel en ylabel, daarom deze hier ook verwijderen.
    g.set(xlabel='', ylabel='')

    # de screenshots van opdracht 2 bevatten ook geen titel voor de legenda, daarom deze hier ook verwijderen.
    plt.legend(title='')

    # de province namen geroteerd voor de leesbaarheid, conform de screenshots van opdracht 2
    plt.xticks(rotation=45)

    plt.show()

def show_province_graph(location, year='2020', hue_order=['Total_reported','Hospital_admission','Deceased'], municipalities=False, show_months=False):
#     province_data = datasets[year].groupby(['Province', 'Type'], numeric_only=True).sum().reset_index()
    province_data = datasets[year]
    province_data = province_data[province_data['Province'] == location]
    if show_months == True:
        province_data = datasets[year].groupby(['Month', 'Type'])['Numbers'].sum().reset_index()
        
        # Gebruik de map-methode van pandas om de maandnummers te vervangen door de corresponderende maandnamen
        province_data['Month'] = province_data['Month'].map(month_names)
        x_axisData = 'Month'
    elif municipalities == False:
        province_data = province_data.groupby(['Province', 'Type', 'Year'])['Numbers'].sum().reset_index()
        x_axisData = 'Province'
    else:
        province_data = province_data.groupby(['Municipality_name', 'Type', 'Year'])['Numbers'].sum().reset_index()
        x_axisData = 'Municipality_name' 
    
   
    # toon de barplot conform de tweede screenshot van opdracht 2
    g = sns.barplot(x = x_axisData,
                    y = 'Numbers',
                    hue = 'Type',
                    hue_order=hue_order,
                    data = province_data)

    # de screenshots van opdracht 2 bevatten geen xlabel en ylabel, daarom deze hier ook verwijderen.
    g.set(xlabel='', ylabel='')

    # de screenshots van opdracht 2 bevatten ook geen titel voor de legenda, daarom deze hier ook verwijderen.
    plt.legend(title='')

    # de province namen geroteerd voor de leesbaarheid, conform de screenshots van opdracht 2
    plt.xticks(rotation=90)

    plt.show()

def show_heatmap(year='2020', hue_order=['Total_reported','Hospital_admission','Deceased'], show_months=False):
    

    
    # Maak een pivot table van de coronadata, zodat elke gemeente een rij is en elke maand een kolom
    corona_pivot = pd.pivot_table(corona_per_gemeente, values='Numbers', index='Municipality_name', columns=['Year', 'Month'])

    # Verwijder de laatste rij (onbekende gemeente)
    corona_pivot = corona_pivot.iloc[:-1]


    # Voeg de coronadata toe aan de gemeentegrenzen
    gemeenten = datasets['gemeenten'].join(corona_pivot)
    
    # Maak een heatmap van het aantal gemelde coronagevallen per gemeente en maand
    sns.heatmap(gemeenten.loc[:, ('2020', slice(None))].sum(axis=1).unstack(level=1), cmap='YlOrRd')

    # Toon de plot
    plt.show()


In [19]:
# deze functie is de brug tussen de gebruikersinvoer en de visuatie die getoond moet worden
def event_handler(year='2020', total_reported=True, hospital_admission=True, deceased=True, grouping='Netherlands', municipalities=False, show_months=False, heatmap=False):
    # Afhankelijk van de gebruikersinvoer, de juiste datasoort (Total_reported, Hospital_admission en Deceased) tonen
    hue_order=[]
    if total_reported:
        hue_order.append('Total_reported')
    if hospital_admission:
        hue_order.append('Hospital_admission')
    if deceased:
        hue_order.append('Deceased')

    # als geen opties zijn geselecteerd, dan alle opties tonen, om Python exception te voorkomen
    if len(hue_order) == 0:
        hue_order=['Total_reported','Hospital_admission','Deceased']
        
    # Afhankelijk van de gebruikersinvoer, de juiste groepering (landelijk of per province) tonen    
    if grouping =='Netherlands':
        show_national_graph(year,hue_order, show_months)
    elif grouping =='All provinces':
        show_provinces_graph(year,hue_order)
    else:
        show_province_graph(grouping, year, hue_order, municipalities, show_months)
    
    if heatmap == True:
        show_heatmap(year, hue_order, show_months)

        
# deze functie toont de GUI componenten en geeft de gebruikersinvoer door aan de event_handler

# dropdown opties om het te tonen jaar te selecteren
year_selector = widgets.Dropdown(
    options=datasets.keys(), # alleen de opties tonen waar datasets beschikbaar voor zijn
    value='2020',
    description='year',
    disabled=False
    )

# checkbox tonen voor Total_reported
total_reported_selector = widgets.Checkbox(
    value=True,
    description='Total_reported',
    disabled=False,
    indent=True
)

# checkbox tonen voor Hospital_admission
hospital_admission_selector = widgets.Checkbox(
    value=True,
    description='Hospital_admission',
    disabled=False,
    indent=True
)

# checkbox tonen voor Deceased
deceased_selector = widgets.Checkbox(
    value=True,
    description='Deceased',
    disabled=False,
    indent=True
)

# dropdown opties om het te tonen groepering te selecteren
provinces = list(dataset_OU['Province'].unique())

grouping_selector = widgets.Dropdown(
    options=['Netherlands','All provinces'] + provinces,
    value='Netherlands',
    description='province',
    disabled=False
    )

municipalities_selector = widgets.Checkbox(
    value=False,
    description='Municipalities',
    disabled=False,
    indent=True,
    layout={'visibility': 'hidden'}
)

show_months_selector = widgets.Checkbox(
    value=False,
    description='Months',
    disabled=False,
    indent=True
)

show_heatmap_selector = widgets.Checkbox(
    value=False,
    description='Heatmap',
    disabled=False,
    indent=True
)

# interactieve dataselectie en datavisualisatie
widgets.interact(event_handler, year=year_selector,
                 total_reported=total_reported_selector,
                 hospital_admission=hospital_admission_selector,
                 deceased=deceased_selector, grouping=grouping_selector,
                 municipalities=municipalities_selector, show_months=show_months_selector,
                heatmap=show_heatmap_selector)


# definieer de functie die wordt uitgevoerd wanneer de "show_months" checkbox wordt aangevinkt of uitgevinkt
# Als show_months op True is, set municipalities op False
def change_months_or_municipalites(change):
    if change['owner'].description == 'Municipalities' and change['owner'].value == True:
        show_months_selector.value = False
    elif change['owner'].description == 'Months' and change['owner'].value == True:
        municipalities_selector.value = False
show_months_selector.observe(change_months_or_municipalites, names='value')
municipalities_selector.observe(change_months_or_municipalites, names='value')

def on_grouping_selector_change(change):
    if grouping_selector.value in ['Netherlands']:
        municipalities_selector.layout.visibility = 'hidden'
        show_months_selector.layout.visibility = 'visible'
    elif grouping_selector.value in ['All provinces']:
        municipalities_selector.layout.visibility = 'hidden'
        show_months_selector.layout.visibility = 'hidden'
    else:
        municipalities_selector.layout.visibility = 'visible'
        show_months_selector.layout.visibility = 'visible'

grouping_selector.observe(on_grouping_selector_change, names='value')


interactive(children=(Dropdown(description='year', options=('2020', '2021', 'gemeenten'), value='2020'), Check…